In [ ]:
from amdex import AMdEX
import hashlib
import json

DATAEXCHANGE_URL = "https://develop.amdex.dev"
TRUST_URL = "https://develop.trust.amdex.dev"

email = "feduser-b@amdex.demo"
password = "yes"
password_hash = hashlib.sha256(password.encode()).hexdigest().upper()

In [ ]:
# Create an instance of AMdEX class (it would make sense to call this trust instead of amdex)
amdex = AMdEX(TRUST_URL)
amdex.signin(email, password_hash)
trust_account_info = amdex.get_session()
print("trust_account_info:", trust_account_info)

trust_version_info = amdex.get_version()
print("trust_version_info:", trust_version_info)

In [ ]:
dataexchange = AMdEX(DATAEXCHANGE_URL)
dataexchange.signin(email, password_hash)
dataexchange_account_info = dataexchange.get_session()
print("dataexchange_account_info:", dataexchange_account_info)

dataexchange_version_info = dataexchange.get_version()
print("dataexchange_version_info:", dataexchange_version_info)

In [ ]:
# # Get dataviews
# dataviews = dataexchange.get_dataviews()
# print("Dataviews:", dataviews)

# # Create a dataview
# dataview_body = {
#     "name": "Sample Dataview",
#     "policy_uuids": ["example_policy_uuid1", "example_policy_uuid2"]
#     # "query": "SELECT * FROM table",
#     # ... other properties
# }
# new_dataview = dataexchange.create_dataview(dataview_body)
# print("New Dataview:", new_dataview)

# print("uuid", new_dataview)

# # Delete a dataview
# uuid = "731ea95e-d608-40db-91ae-fae98a8c0794"
# result = dataexchange.delete_dataview(new_dataview)
# print("Delete Result:", result)

In [ ]:
#create an amdex class for the compliance officer
email_complianceofficer = "complianceofficer@amdex.demo"
password_complianceofficer = "yes"
password_hash_complianceofficer = hashlib.sha256(password_complianceofficer.encode()).hexdigest().upper()


# # Create an instance of AMdEX class
amdex_complianceofficer = AMdEX()
amdex_complianceofficer.signin(email_complianceofficer, password_hash_complianceofficer)

account_info_complianceofficer = amdex_complianceofficer.get_session()
print("account info compliance officer:", account_info_complianceofficer)



In [ ]:
# Get policies
print("6. Get policies:")
policies = amdex_complianceofficer.get_policies()


print(f"Got {policies.__len__()} policies")
print("Policies:")
for policy in policies:
    print(f"Name: {policy['name']}\nUUID: {policy['policy_uuid']}\n")
print()

# Create policy
print("7. Create policy:")
policy_body = {
    "name": "Example policy 1",
    "inputs": [
        {
            "description": "Consortium UUID of data provider",
            "name": "provider_consortium_uuid",
            "source": "internal"
        },
        {
            "description": "Consortium UUID of data consumer",
            "name": "consumer_consortium_uuid",
            "source": "internal"
        }
    ],
    "outputs": [
        {
            "description": "Access allowed",
            "name": "access_allowed"
        }
    ],
    "url": {
        "human": "https://amdex.dev/policies/gdpr-attribution/README.txt",
        "legal": "https://amdex.dev/policies/gdpr-attribution/POLICY.txt",
        "machine": "https://amdex.dev/policies/gdpr-attribution/gdpr-attribution.eflint"
    },
    "hash_type": "SHA-256",
    "hash": {
        "human": "e5e61f918d66cd421ee506dff43f713b3e74449a23d035b65b8115196d7397d9",
        "legal": "d0119b1ff66c27f3973c0437bc2a19d426f6a5532313abca9cd9b7fdd37ba614",
        "machine": "8C2B4CA076E57619E239958200FF18D4CFC0E931755C2648D66B36F2BD4D7E0B"
    }
}

created_policy = amdex_complianceofficer.create_policy(policy_body)
print("Created Policy:", created_policy)
print()

# Delete policy
# print("8. Delete policy:")
# policy_uuid = created_policy.get("policy_uuid")
# deleted_policy = amdex_complianceofficer.delete_policy(policy_uuid)
# print("Deleted Policy:", deleted_policy)
# print()

In [ ]:
# policies
print("6. Get policies:")
policies = amdex_complianceofficer.get_policies()
print("Policies:", policies)
print()

In [ ]:
# Get jobs
print("9. Get jobs:")
jobs = amdex.get_jobs()
print("10. Existing Jobs:", jobs)

In [ ]:
print("11. Create Parent job:")
parent_job_body = {
    "type": "create",
    "name": "Jupyter Notebook Parent Job",
    "data": { "policy_uuids": ["21693473-d070-43bf-aca7-b75adbb51e48", "8cceff93-ac99-4d10-a121-35413e3bea5a"] },
    "context": {
        "provider_consortium_uuid": "Parent job provider_consortium_uuid",
        "consumer_consortium_uuid": "Parent job consumer_consortium_uuid",
        "dataview_uuid": "Parent job dataview_uuid",
        "a_pip_url_from_context": DATAEXCHANGE_URL + "/api/pip"
    }
}
parent_job = amdex.create_job(parent_job_body)
parent_job_uuid = parent_job['job_uuid']
print("12. Parent Job:", parent_job)

print("13. Poll parent job status:")
parent_job_status = amdex.poll_job_status(parent_job_uuid)
print("14. Parent Job Status:", parent_job_status)
print()

In [ ]:
print("15. Create another job with our newly created policies and mark the previous job as parent:")
child_job_body = {
    "type": "create",
    "name": "Jupyter Notebook Child Job",
    "parent_job_uuid": parent_job_uuid,
    "data": { "policy_uuids": [created_policy['policy_uuid'], created_policy['policy_uuid'], created_policy['policy_uuid']] },
    "context": { 
        "provider_consortium_uuid": "Child job provider_consortium_uuid",
        "consumer_consortium_uuid": "Child job consumer_consortium_uuid"
    }
}
child_job = amdex.create_job(child_job_body)
child_job_uuid = child_job['job_uuid']

print("16. Child Job:", child_job)

print("17. Poll child job status:")
child_job_status = amdex.poll_job_status(child_job_uuid)

print("18. Child Job Status:", child_job_status)
print()

In [ ]:
# Delete all jobs
# print("18. Delete jobs:")
# deleted_jobs = amdex.delete_jobs()
# print("Deleted Jobs:", deleted_jobs)
# print()